In [1]:
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
%matplotlib notebook
from sklearn import svm, metrics, datasets
from sklearn.utils import Bunch
from sklearn.model_selection import GridSearchCV, train_test_split

from skimage.io import imread
from skimage.transform import resize

In [2]:
def load_image_files(container_path, dimension=(64, 64)):
    """
    Load image files with categories as subfolder names 
    which performs like scikit-learn sample dataset
    
    Parameters
    ----------
    container_path : string or unicode
        Path to the main folder holding one subfolder per category
    dimension : tuple
        size to which image are adjusted to
        
    Returns
    -------
    Bunch
    """
    image_dir = Path(container_path)
    folders = [directory for directory in image_dir.iterdir() if directory.is_dir()]
    categories = [fo.name for fo in folders]

    descr = "A image classification dataset"
    images = []
    flat_data = []
    target = []
    for i, direc in enumerate(folders):
        for file in direc.iterdir():
            img = imread(file)
            img_resized = resize(img, dimension, anti_aliasing=True, mode='reflect')
            flat_data.append(img_resized.flatten()) 
            images.append(img_resized)
            target.append(i)
    flat_data = np.array(flat_data)
    target = np.array(target)
    images = np.array(images)

    return Bunch(data=flat_data,
                 target=target,
                 target_names=categories,
                 images=images,
                 DESCR=descr)

In [3]:
image_dataset = load_image_files("images/")

In [22]:
X_train, X_test, y_train, y_test = train_test_split(
    image_dataset.data, image_dataset.target, test_size=0.4,random_state=109)

In [28]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier( max_iter=100)
parameter_space = {
    'hidden_layer_sizes': [(64,)],
    'activation': ['relu','tanh'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['adaptive','constant'],
}

from sklearn.model_selection import GridSearchCV

clf = GridSearchCV(mlp, parameter_space, n_jobs=-1, cv=5)
clf.fit(X_train, y_train) # X is train samples and y is the corresponding labels



C:\Users\jallu\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


GridSearchCV(cv=5, estimator=MLPClassifier(max_iter=100), n_jobs=-1,
             param_grid={'activation': ['relu', 'tanh'],
                         'alpha': [0.0001, 0.05], 'hidden_layer_sizes': [(64,)],
                         'learning_rate': ['adaptive'],
                         'solver': ['sgd', 'adam']})

In [29]:
print('Best parameters found:\n', clf.best_params_)

Best parameters found:
 {'activation': 'tanh', 'alpha': 0.05, 'hidden_layer_sizes': (64,), 'learning_rate': 'adaptive', 'solver': 'sgd'}


In [30]:
#mlp.fit(X_train,y_train)

predict_train = clf.predict(X_train)
predict_test = clf.predict(X_test)

In [31]:
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_train,predict_train))
print(classification_report(y_train,predict_train))

[[37  1  0  0  0]
 [ 1 31  0  0  0]
 [ 0  0 32  0  0]
 [ 0  2  0 26  0]
 [ 0  0  1  0 54]]
              precision    recall  f1-score   support

           0       0.97      0.97      0.97        38
           1       0.91      0.97      0.94        32
           2       0.97      1.00      0.98        32
           3       1.00      0.93      0.96        28
           4       1.00      0.98      0.99        55

    accuracy                           0.97       185
   macro avg       0.97      0.97      0.97       185
weighted avg       0.97      0.97      0.97       185



In [32]:
print(confusion_matrix(y_test,predict_test))
print(classification_report(y_test,predict_test))

[[17  4  2  5  1]
 [ 2 15  1  1  0]
 [ 1  0 16  0  3]
 [11  1  2 11  1]
 [ 0  0  0  0 30]]
              precision    recall  f1-score   support

           0       0.55      0.59      0.57        29
           1       0.75      0.79      0.77        19
           2       0.76      0.80      0.78        20
           3       0.65      0.42      0.51        26
           4       0.86      1.00      0.92        30

    accuracy                           0.72       124
   macro avg       0.71      0.72      0.71       124
weighted avg       0.71      0.72      0.71       124

